In [1]:
import enerhabitat as eh
import pandas as pd 
import plotly.express as px
import time

eh.materials('./eh_config/materials.ini')

sc = [
    ("Adobe", 0.1),
    ("Acero", 0.1),
#    ("brick", 0.2),
#    ("concrete", 0.1),
]

In [2]:
t0 = time.perf_counter()

dia_promedio = eh.meanDay(epw_file="epw/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw")

t1 = time.perf_counter()
print(f"Tardó {(t1 - t0)*1000:.3f} ms\n")

dia_promedio.info()

Tardó 758.744 ms

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 86341 entries, 2025-10-15 00:00:00-08:00 to 2025-10-15 23:59:00-08:00
Freq: s
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zenith            86341 non-null  float64
 1   elevation         86341 non-null  float64
 2   azimuth           86341 non-null  float64
 3   equation_of_time  86341 non-null  float64
 4   Ta                86341 non-null  float64
 5   Ig                86341 non-null  float64
 6   Ib                86341 non-null  float64
 7   Id                86341 non-null  float64
 8   Tn                86341 non-null  float64
 9   DeltaTn           86341 non-null  float64
dtypes: float64(10)
memory usage: 7.2 MB


In [3]:
t0 = time.perf_counter()

tsa_df = eh.Tsa(
    meanDay_dataframe=dia_promedio,
    solar_absortance=0.8,
    surface_tilt=90, 
    surface_azimuth=90,
    )

t1 = time.perf_counter()
print(f"Tardó {(t1 - t0)*1000:.3f} ms\n")

tsa_df.info()

Tardó 11.024 ms

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 86341 entries, 2025-10-15 00:00:00-08:00 to 2025-10-15 23:59:00-08:00
Freq: s
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zenith            86341 non-null  float64
 1   elevation         86341 non-null  float64
 2   azimuth           86341 non-null  float64
 3   equation_of_time  86341 non-null  float64
 4   Ta                86341 non-null  float64
 5   Ig                86341 non-null  float64
 6   Ib                86341 non-null  float64
 7   Id                86341 non-null  float64
 8   Tn                86341 non-null  float64
 9   DeltaTn           86341 non-null  float64
 10  Is                86341 non-null  float64
 11  Tsa               86341 non-null  float64
dtypes: float64(12)
memory usage: 8.6 MB


In [4]:
t0 = time.perf_counter()

sc_df = eh.solveCS(sc, tsa_df)

t1 = time.perf_counter()
print(f"Tardó {(t1 - t0)*1000:.3f} ms\n")

sc_df.info()
sc_df = sc_df.round(2)

Tardó 40.501 ms

<class 'pandas.core.series.Series'>
DatetimeIndex: 144 entries, 2025-10-15 00:00:00-08:00 to 2025-10-15 23:50:00-08:00
Freq: 600s
Series name: Ti
Non-Null Count  Dtype  
--------------  -----  
144 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB


In [5]:
resultado = tsa_df.join(sc_df, how="right")
resultado.info()

solucion = pd.concat([resultado[['zenith', 'elevation', 'azimuth', 'equation_of_time', 'Tn', 'DeltaTn', 'Ta', 'Ig', 'Ib', 'Id', 'Is']], resultado[["Tsa", "Ti"]].add_suffix("_1", axis=1)], axis=1)
solucion.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 2025-10-15 00:00:00-08:00 to 2025-10-15 23:50:00-08:00
Freq: 600s
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zenith            144 non-null    float64
 1   elevation         144 non-null    float64
 2   azimuth           144 non-null    float64
 3   equation_of_time  144 non-null    float64
 4   Ta                144 non-null    float64
 5   Ig                144 non-null    float64
 6   Ib                144 non-null    float64
 7   Id                144 non-null    float64
 8   Tn                144 non-null    float64
 9   DeltaTn           144 non-null    float64
 10  Is                144 non-null    float64
 11  Tsa               144 non-null    float64
 12  Ti                144 non-null    float64
dtypes: float64(13)
memory usage: 15.8 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 2025-10-15 00:00:00-08

In [6]:
px.scatter(
    data_frame=solucion,
    x=solucion.index,
    y=["Ta","Tsa_1","Ti_1"]
)